# Load Test deployed web application

This notebook pulls some images and tests them against the deployed web application. We submit requests asychronously which should reduce the contribution of latency.

In [ ]:
import os
from timeit import default_timer
import pandas as pd

import matplotlib.pyplot as plt
from azureml.core.webservice import AksWebservice
from azureml.core.workspace import Workspace
from dotenv import get_key, find_dotenv
from testing_utilities import to_img, gen_variations_of_one_image, get_auth
from urllib.parse import urlparse

%matplotlib inline

In [ ]:
env_path = find_dotenv(raise_error_if_not_found=True)

In [ ]:
ws = Workspace.from_config(auth=get_auth())
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep="\n")

Let's retrive the web service.

In [ ]:
aks_service_name = get_key(env_path, 'aks_service_name')
aks_service = AksWebservice(ws, name=aks_service_name)

We will test our service concurrently but only have 4 concurrent requests at any time. We have only deployed one pod on one node and increasing the number of concurrent calls does not really increase throughput. Feel free to try different values and see how the service responds.

In [ ]:
CONCURRENT_REQUESTS = 4   # Number of requests at a time

Get the scoring URL and API key of the service.

In [ ]:
scoring_url = aks_service.scoring_uri
api_key = aks_service.get_keys()[0]

In [ ]:
IMAGEURL = "https://bostondata.blob.core.windows.net/aksdeploymenttutorialaml/220px-Lynx_lynx_poing.jpg"
plt.imshow(to_img(IMAGEURL))

Below we are going to use [Locust](https://locust.io/) to load test our deployed model. First we need to write the locustfile. We will use variations of the same image to test the service.

In [ ]:
%%writefile locustfile.py
from locust import HttpLocust, TaskSet, task
from testing_utilities import gen_variations_of_one_image
import os
from itertools import cycle


_IMAGEURL = os.getenv('IMAGEURL', "https://bostondata.blob.core.windows.net/aksdeploymenttutorialaml/220px-Lynx_lynx_poing.jpg")
_NUMBER_OF_VARIATIONS = os.getenv('NUMBER_OF_VARIATIONS', 100)
_SCORE_PATH = os.getenv('SCORE_PATH', "/score")
_API_KEY = os.getenv('API_KEY')


class UserBehavior(TaskSet):
    def on_start(self):
        print('Running setup')
        self._image_generator =  cycle(gen_variations_of_one_image(_IMAGEURL, _NUMBER_OF_VARIATIONS))
        self._headers = {'Authorization':('Bearer {}'.format(_API_KEY))}
        
    @task
    def score(self):
        self.client.post(_SCORE_PATH, files={'image': next(self._image_generator)}, headers=self._headers)


class WebsiteUser(HttpLocust):
    task_set = UserBehavior
    # min and max time to wait before repeating task
    min_wait = 10
    max_wait = 200

Below we define the locust command we want to run. We are going to run at a hatch rate of 10 and the whole test will last 1 minute. Feel free to adjust the parameters below and see how the results differ. The results of the test will be saved to two csv files **modeltest_requests.csv** and **modeltest_distribution.csv**

In [ ]:
parsed_url = urlparse(scoring_url)
cmd = "locust -H {host} --no-web -c {users} -r {rate} -t {duration} --csv=modeltest --only-summary".format(
    host="{url.scheme}://{url.netloc}".format(url=parsed_url),
    users=CONCURRENT_REQUESTS,  # concurrent users
    rate=10,                    # hatch rate (users / second)
    duration='1m',              # test duration
)

In [ ]:
! API_KEY={api_key} SCORE_PATH={parsed_url.path} PYTHONPATH={os.path.abspath('../')} {cmd}

Here are the summary results of our test and below that the distribution infromation of those tests. 

In [ ]:
pd.read_csv("modeltest_requests.csv")

In [ ]:
pd.read_csv("modeltest_distribution.csv")

To tear down the cluster and all related resources go to the [tear down the cluster](07_TearDown.ipynb) notebook.